In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os 
import sys

project_dir_path = '/home/onoue/ws/lukasiewicz_1'
sys.path.append(project_dir_path)

for path in sys.path:
    print(path)

from src.setup_problem import Setup
from src.misc import visualize_result

import cvxpy as cp

from src.misc import *

/home/onoue/ws/lukasiewicz_1/notebooks/pima_indian_diabetes
/usr/lib/python310.zip
/usr/lib/python3.10
/usr/lib/python3.10/lib-dynload

/home/onoue/ws/lukasiewicz_1/myenv/lib/python3.10/site-packages
/home/onoue/ws/lukasiewicz_1


# データの準備

In [16]:
# data_cleaned = pd.read_csv('./data/diabetes_cleaned.csv', index_col=0)
# display(data_cleaned)

In [21]:
data = pd.read_csv('./data/diabetes_discretized.csv', index_col=0)
# display(data)

In [23]:
outcome = data['Outcome']
# display(outcome)

features = data.drop(['Outcome'], axis=1)
# display(features)

feature_names = list(features.columns)
feature_names

['Pregnancies_Low',
 'Pregnancies_Medium',
 'Pregnancies_High',
 'Glucose_Low',
 'Glucose_Medium',
 'Glucose_High',
 'BloodPressure_Low',
 'BloodPressure_Medium',
 'BloodPressure_High',
 'SkinThickness_Low',
 'SkinThickness_Medium',
 'SkinThickness_High',
 'BMI_Low',
 'BMI_Medium',
 'BMI_High',
 'DiabetesPedigreeFunction_Low',
 'DiabetesPedigreeFunction_Medium',
 'DiabetesPedigreeFunction_High',
 'Age_Low',
 'Age_Medium',
 'Age_High']

In [29]:
data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')

if not os.path.exists(data_dir_path):
    os.mkdir(data_dir_path)

In [30]:
df = features.copy()
df['target'] = outcome
display(df)

file_name = "L_" + "Outcome"
file_path = os.path.join(data_dir_path, file_name)
df.to_csv(file_path)

,Pregnancies_Low,Pregnancies_Medium,Pregnancies_High,Glucose_Low,Glucose_Medium,Glucose_High,BloodPressure_Low,BloodPressure_Medium,BloodPressure_High,SkinThickness_Low,...,BMI_Low,BMI_Medium,BMI_High,DiabetesPedigreeFunction_Low,DiabetesPedigreeFunction_Medium,DiabetesPedigreeFunction_High,Age_Low,Age_Medium,Age_High,target
0,0,1,0,0,0,1,0,1,0,0,...,0,1,0,0,1,0,0,1,0,1
1,1,0,0,1,0,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,0
2,0,1,0,0,0,1,0,1,0,1,...,1,0,0,0,1,0,1,0,0,1
3,1,0,0,1,0,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,0
5,0,1,0,0,1,0,0,1,0,1,...,1,0,0,1,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
763,0,0,1,0,1,0,0,1,0,0,...,0,1,0,1,0,0,0,0,1,0
764,1,0,0,0,1,0,0,1,0,0,...,0,1,0,1,0,0,1,0,0,0
765,0,1,0,0,1,0,0,1,0,0,...,1,0,0,1,0,0,1,0,0,0
766,1,0,0,0,1,0,1,0,0,1,...,0,1,0,1,0,0,0,1,0,1


In [33]:
for feature_name in feature_names:
    df = features.copy()
    df['target'] = df[feature_name]
    # display(df)

    file_name = "L_" + feature_name
    file_path = os.path.join(data_dir_path, file_name)
    df.to_csv(file_path)

# 

In [35]:
data_dir_path = os.path.join(project_dir_path, 'inputs/pima_indian_diabetes')
data_dir_path

'/home/onoue/ws/lukasiewicz_1/inputs/pima_indian_diabetes'

In [36]:
os.listdir(data_dir_path)

['L_Outcome',
 'L_DiabetesPedigreeFunction_Medium',
 'train_index.csv',
 'L_BMI_Medium',
 'L_BMI_Low',
 'L_Pregnancies_Low',
 'rules.txt',
 'L_SkinThickness_High',
 'L_BloodPressure_Low',
 'L_SkinThickness_Low',
 'L_BloodPressure_Medium',
 'L_SkinThickness_Medium',
 'rules_max_50.txt',
 'L_Age_Low',
 'L_Glucose_Medium',
 'L_DiabetesPedigreeFunction_Low',
 'L_Glucose_High',
 'L_Pregnancies_Medium',
 'L_BloodPressure_High',
 'test_index.csv',
 'L_BMI_High',
 'L_Age_Medium',
 'L_Glucose_Low',
 'L_DiabetesPedigreeFunction_High',
 'L_Pregnancies_High',
 'L_Age_High']

In [39]:
train_index = pd.read_csv(os.path.join(data_dir_path, "train_index.csv"), index_col=0)
display(train_index)

test_index = pd.read_csv(os.path.join(data_dir_path, "test_index.csv"), index_col=0)
display(test_index)

,0
0,275
1,555
2,699
3,73
4,170
...,...
531,87
532,128
533,315
534,505


,0
0,419
1,186
2,556
3,738
4,320
...,...
129,626
130,487
131,638
132,248


In [ ]:
test_index

In [8]:
data_winston = pd.read_csv('./../winston/data/data.csv', index_col=0)
labels_winston = pd.read_csv('./../winston/data/labels.csv', index_col=0)

display(data_winston)
display(labels_winston)

,folder,name,width,height,depth,R,G,B
0,penguin,8065,1200,1600,3,0.486640,0.440085,0.342231
1,penguin,8610,468,500,3,0.427333,0.406967,0.376660
2,penguin,8445,500,375,3,0.509321,0.512104,0.398580
3,penguin,1188,85,62,3,0.581645,0.572609,0.549550
4,penguin,8237,375,500,3,0.505475,0.517679,0.422280
...,...,...,...,...,...,...,...,...
6495,zebra,2311,333,500,3,0.618658,0.583356,0.521907
6496,zebra,6650,379,500,3,0.506803,0.401962,0.288625
6497,zebra,2627,500,375,3,0.517260,0.509384,0.498286
6498,zebra,7258,375,500,3,0.470308,0.439706,0.373938


,albatross,cheetah,ostrich,penguin,zebra
hair(x),NaN,1.0,NaN,NaN,1.0
mammal(x),NaN,1.0,NaN,NaN,1.0
milk(x),NaN,1.0,NaN,NaN,NaN
feather(x),1.0,NaN,1.0,1.0,NaN
bird(x),1.0,NaN,1.0,1.0,NaN
layeggs(x),1.0,NaN,1.0,1.0,NaN
meat(x),NaN,1.0,NaN,NaN,NaN
carnivore(x),NaN,1.0,NaN,NaN,NaN
pointedteeth(x),NaN,1.0,NaN,NaN,NaN
claws(x),NaN,1.0,NaN,NaN,NaN
